In [1]:
from agent import QLearningAgent
from verify_data import FishGame, ParseError

In [2]:
import os

memories = []
for filename in os.listdir('data'):
    filepath = os.path.join('data', filename)
    with open(filepath, 'r') as f:
        try:
            print(f"{filename}")
            game = FishGame(f.readlines())
            for player in game.players:
                memories += game.memory(player)
        except ParseError as e:
            print(f"{filename}: {e}")
            break

12-3_15:27.txt
12-10_11:07.txt
12-3_14:05.txt
12-3_11:30.txt
12-3_14:27.txt
12-6_11:08.txt
12-3_11:12.txt
12-4_11:11.txt
1-15_11:15.txt


In [3]:
split = round(len(memories) * .8)
train_memories = memories[:split]
test_memories = memories[split:]

In [4]:
agent = QLearningAgent()
agent.train_on_data(train_memories, test_memories, 20)

/Users/Yourui/Documents/Fish/.env/lib/python3.13/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/Users/Yourui/Documents/Fish/agent.py:76: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:257.)
  return torch.FloatTensor(x).to(self.device)
/Users/Yourui/Documents/Fish/agent.py:51: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  'ask_card': F.softmax(ask_card),


epoch 0, train loss 0.04725, test loss 0.03354, lr 0.001
epoch 1, train loss 0.03483, test loss 0.02885, lr 0.001
epoch 2, train loss 0.03211, test loss 0.03778, lr 0.001
epoch 3, train loss 0.02985, test loss 0.02896, lr 0.001
epoch 4, train loss 0.03288, test loss 0.03621, lr 0.001
epoch 5, train loss 0.02859, test loss 0.01978, lr 0.001
epoch 6, train loss 0.03128, test loss 0.01878, lr 0.001
epoch 7, train loss 0.02241, test loss 0.03685, lr 0.001
epoch 8, train loss 0.02132, test loss 0.03065, lr 0.001
epoch 9, train loss 0.02441, test loss 0.03095, lr 0.001
epoch 10, train loss 0.02258, test loss 0.02919, lr 0.001
epoch 11, train loss 0.02067, test loss 0.03244, lr 0.001
epoch 12, train loss 0.02115, test loss 0.02941, lr 0.001
epoch 13, train loss 0.01843, test loss 0.03253, lr 0.001
epoch 14, train loss 0.01665, test loss 0.02964, lr 0.001
epoch 15, train loss 0.01596, test loss 0.02973, lr 0.001
epoch 16, train loss 0.01584, test loss 0.02965, lr 0.001
epoch 17, train loss 0.0

In [7]:
import os
from datetime import datetime

os.makedirs('models', exist_ok=True)
model_path = f'models/fish_agent_{datetime.now().strftime("%Y%m%d_%H%M%S")}.pth'
agent.save_model(model_path)

Model saved to models/fish_agent_20250325_203048.pth


In [8]:
agent = QLearningAgent() 
agent.load_model('models/fish_agent_20250319_123306.pth')

True

In [9]:
game = FishGame(open('data/1-15_11:15.txt', 'r').readlines())
player = 'PB'
test_data = agent.unpack_batch(game.memory(player)[1:2])
agent.q_network(agent.tensor(test_data['state']), 
                agent.action_masks(*test_data['mask_dep'].values()))

{'call': tensor([[0.3735, 0.6265]], device='mps:0', grad_fn=<SoftmaxBackward0>),
 'call_set': tensor([[0.0683, 0.3328, 0.1711, 0.0416, 0.1856, 0.0393, 0.1124, 0.0488, 0.0000]],
        device='mps:0', grad_fn=<SoftmaxBackward0>),
 'call_cards': tensor([[[0.2083, 0.2714, 0.2998, 0.2205],
          [0.1354, 0.2199, 0.3274, 0.3173],
          [0.4047, 0.3686, 0.0762, 0.1506],
          [0.1540, 0.3419, 0.0747, 0.4293],
          [0.3549, 0.1006, 0.3225, 0.2219],
          [0.1586, 0.2986, 0.3635, 0.1793]]], device='mps:0',
        grad_fn=<SoftmaxBackward0>),
 'ask_person': tensor([[0.2090, 0.1333, 0.6202, 0.0376]], device='mps:0',
        grad_fn=<SoftmaxBackward0>),
 'ask_set': tensor([[0.0000, 0.1338, 0.0000, 0.6462, 0.0475, 0.1345, 0.0380, 0.0000, 0.0000]],
        device='mps:0', grad_fn=<SoftmaxBackward0>),
 'ask_card': tensor([[2.7193e-01, 9.0935e-02, 6.2667e-01, 3.1250e-06, 1.6242e-06, 1.0456e-02]],
        device='mps:0', grad_fn=<SoftmaxBackward0>)}